In [15]:
######################
# Process MIDI Files #
######################

from music21 import converter
from music21 import instrument
from music21 import chord
from music21 import note
import glob
import json
import os

# VARS
DATA_GLOB = "data/pokemon/PkmRB/*.mid"
MODEL_DIR = "model/pokemon/PkmRB-V1"
NOTES_FILE = "%s/%s" % (MODEL_DIR, "notes.json")

NOTES_BY_FILE = []
NOTES = set()

for input_file in glob.glob(DATA_GLOB):
    print("Ingesting... %s" % input_file)
    
    midi = converter.parse(input_file)
    parts = instrument.partitionByInstrument(midi)
    if parts:
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
    
    file_notes = []
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            file_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            file_notes.append('.'.join(str(n) for n in element.normalOrder))
    NOTES_BY_FILE.append(file_notes)
    NOTES.update(set(file_notes))

NOTES = sorted(NOTES)

if not os.path.exists(os.path.dirname(NOTES_FILE)):
    os.makedirs(os.path.dirname(NOTES_FILE))
with open(NOTES_FILE, 'w') as note_file:
    json.dump(NOTES, note_file)

Ingesting... data/pokemon/PkmRB/PkmRB-Battle3.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Evolving.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Celadon.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Victory1.mid
Ingesting... data/pokemon/PkmRB/PkmRB-10.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Pallet.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Caught.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Cinnabar.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Battle1.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Victory2.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Enc1.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Rival.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Lavender.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Mansion.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Hideout.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Pokedex.mid
Ingesting... data/pokemon/PkmRB/PkmRB-24.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Vermilion.mid
Ingesting... data/pokemon/PkmRB/PkmRB-12.mid
Ingesting... data/pokemon/PkmRB/PkmRB-1.mid
Ingesting... data/pok

In [17]:
########################
# Build Note Sequences #
########################

import numpy
from keras.utils import np_utils

NOTES_FILE = None
SEQUENCE_LENGTH = 100
X = []
y = []

# Load Notes from disk if desired
if NOTES_FILE:
    with open(NOTES_FILE, 'r') as notes_file:
        NOTES = json.loads(notes_file.read())

NOTE_TO_INT = dict((note, number) for number, note in enumerate(NOTES))
INT_TO_NOTE = dict((number, note) for number, note in enumerate(NOTES))

for notes in NOTES_BY_FILE:
    for i in range(0, len(notes) - SEQUENCE_LENGTH, 1):
        input_sequence = notes[i:i + SEQUENCE_LENGTH] # N Input Notes
        output_sequence = notes[i + SEQUENCE_LENGTH] # 1 Output Note
        
        X.append([NOTE_TO_INT[c] for c in input_sequence])
        y.append(NOTE_TO_INT[output_sequence])
        
N_SEQUENCES = len(X)
N_VOCAB = len(NOTES)

print("Num Sequences: %d" % N_SEQUENCES)
print("Num Vocab: %d" % N_VOCAB)

X = numpy.reshape(X, (N_SEQUENCES, SEQUENCE_LENGTH, 1)) / N_VOCAB
y = np_utils.to_categorical(y)

Num Sequences: 24664
Num Vocab: 56


In [18]:
#######################
# Construct the Model #
#######################

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

CHECKPOINT_FILE = None

model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X.shape[1], X.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(N_VOCAB))
model.add(Activation('softmax'))
if (CHECKPOINT_FILE):
    model.load_weights(CHECKPOINT_FILE)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
###################
# Train the Model #
###################

from keras.callbacks import ModelCheckpoint

filepath = "%s/%s" % (MODEL_DIR, 'weights-improvement-{epoch:02d}-{loss:.4f}.hdf5')
checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=True)
callbacks_list = [checkpoint]

model.fit(X, y, epochs=200, batch_size=128, callbacks=callbacks_list)

Epoch 1/200
24664/24664 [==============================] - 40s 2ms/step - loss: 3.4377
Epoch 2/200
24664/24664 [==============================] - 39s 2ms/step - loss: 3.2260
Epoch 3/200
24664/24664 [==============================] - 40s 2ms/step - loss: 3.0966
Epoch 4/200
24664/24664 [==============================] - 40s 2ms/step - loss: 2.9462
Epoch 5/200
24664/24664 [==============================] - 39s 2ms/step - loss: 2.8097
Epoch 6/200
24664/24664 [==============================] - 39s 2ms/step - loss: 2.6670
Epoch 7/200
24664/24664 [==============================] - 39s 2ms/step - loss: 2.5319
Epoch 8/200
24664/24664 [==============================] - 39s 2ms/step - loss: 2.3971
Epoch 9/200
24664/24664 [==============================] - 39s 2ms/step - loss: 2.2799
Epoch 10/200
24664/24664 [==============================] - 39s 2ms/step - loss: 2.1730
Epoch 11/200
24664/24664 [==============================] - 39s 2ms/step - loss: 2.0745
Epoch 12/200
24664/24664 [===============

24664/24664 [==============================] - 41s 2ms/step - loss: 0.1776
Epoch 95/200
24664/24664 [==============================] - 41s 2ms/step - loss: 0.1725
Epoch 96/200
24664/24664 [==============================] - 41s 2ms/step - loss: 0.1746
Epoch 97/200
24664/24664 [==============================] - 41s 2ms/step - loss: 0.1794
Epoch 98/200
24664/24664 [==============================] - 41s 2ms/step - loss: 0.2466
Epoch 99/200
24664/24664 [==============================] - 41s 2ms/step - loss: 0.1879
Epoch 100/200
24664/24664 [==============================] - 41s 2ms/step - loss: 0.1779
Epoch 101/200
24664/24664 [==============================] - 41s 2ms/step - loss: 0.1696
Epoch 102/200
24664/24664 [==============================] - 40s 2ms/step - loss: 0.1717
Epoch 103/200
24664/24664 [==============================] - 40s 2ms/step - loss: 0.1803
Epoch 104/200
24664/24664 [==============================] - 40s 2ms/step - loss: 0.1723
Epoch 105/200
24664/24664 [=============